# Avaliando o impacto de uma política pública

## Programa de Subsídios a Planos de Saúde (*Health Insurance Subsidy Program - HISP*)

Objetivo primário do programa: reduzir a despesa com itens relacionados a saúde para domicílios de baixa renda.

Variável de interesse: *health_expenditures*

----

## Leitura dos Dados

In [1]:
## Atualizando scipy
from IPython.display import clear_output # limpa o output de uma célula
!pip uninstall scipy -y
!pip install scipy

clear_output()  # limpando o texto

In [2]:
## Já instalando bibliotecas necessárias
!pip install linearmodels

clear_output()  # limpando o texto

In [3]:
## Importando o que for necessário
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import matplotlib as mpl

In [4]:
## Módulos de teste
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import het_breuschpagan, linear_reset
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from patsy import dmatrices

In [5]:
## Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
## Lendo os dados
sCaminho = "/content/drive/Othercomputers/Meu modelo MacBook Air/Documents/IPEA/Curso IDP/Disciplina 2 - Avaliação de Impacto de Políticas - World Bank/Dados/"
sArquivo = "evaluation.dta"

## Lendo o DataFrame
df = pd.read_stata(f"{sCaminho}{sArquivo}")

# Vendo o tamanho da base
print(f"Linhas: {df.shape[0]}; Colunas: {df.shape[1]}")

Linhas: 19827; Colunas: 22


In [8]:
## Vendo o DataFrame
df.head(10)

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
0,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,0.0,15.185455,24.0,23.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,25.0,24.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
2,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,0.0,13.076257,30.0,26.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,31.0,27.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,59.0,57.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0
5,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,0.0,15.286353,58.0,56.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,0.0
6,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,1.0,20.026909,36.0,25.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
7,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,0.0,11.311761,35.0,24.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
8,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,0.0,11.223912,37.0,35.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
9,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,1.0,16.664686,39.0,36.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0


In [9]:
## Vendo a descrição das variáveis (só funciona com arquivos .dta)
# Lendo novamente a base, mas agora com iterator, o que retorna um objeto do tipo StataReader
stata_reader = pd.read_stata(f"{sCaminho}{sArquivo}", iterator=True)

# Pegando as descrições das variáveis
descricoes = stata_reader.variable_labels()
descricoes

{'age_hh': 'Age of the head of the household (in years)',
 'age_sp': 'Age of the spouse (in years)',
 'bathroom': 'Home with private bathroom at baseline (0=no, 1=yes)',
 'dirtfloor': 'Home has a dirt floor at baseline (0=no, 1=yes)',
 'educ_hh': 'Education of the head of household (completed years of schooling)',
 'educ_sp': 'Education of the spouse (completed years of schooling)',
 'eligible': 'Household eligible to enroll in HISP (0=no, 1=yes)',
 'enrolled': 'HH enrolled in HISP (0=no, 1=yes)',
 'enrolled_rp': 'Household enrolled in HISP under the random promotion scenario (0=no, 1=yes)',
 'female_hh': 'Head of the household is a woman (0=no, 1=yes)',
 'health_expenditures': 'Out of pocket health expenditures (per person per year)',
 'hhsize': 'Number of household members (baseline)',
 'hospital': 'HH member visited hospital in the past year (0=no, 1=yes)',
 'hospital_distance': 'Distance to closest hospital',
 'household_identifier': 'Unique household identifier',
 'indigenous': 'H

In [10]:
## Descrevendo o DataFrame
df.describe()

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
count,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,11257.000000
mean,73.933472,8038.965820,0.500277,0.512685,0.567761,0.299037,0.293287,56.789505,0.500025,17.035103,46.683117,40.581734,2.833550,2.618601,0.099057,0.352903,5.178645,0.603621,0.615978,2.079740,105.322517,0.052323
std,55.076599,4569.468262,0.500017,0.499851,0.495377,0.457822,0.455238,10.686106,0.500013,9.291589,15.294811,12.822810,2.754772,2.543370,0.298730,0.477869,2.195178,0.489157,0.486375,3.133202,42.063479,0.222696
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.479134,0.000000,0.000000,14.000000,14.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,9.465392,0.000000
25%,30.000000,3989.500000,0.000000,0.000000,0.000000,0.000000,0.000000,49.652241,0.000000,11.621977,34.000000,31.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,72.804218,0.000000
50%,59.000000,8040.000000,1.000000,1.000000,1.000000,0.000000,0.000000,56.414219,1.000000,16.051678,45.000000,41.000000,2.000000,2.000000,0.000000,0.000000,5.000000,1.000000,1.000000,1.000000,113.556497,0.000000
75%,112.000000,12033.000000,1.000000,1.000000,1.000000,1.000000,1.000000,62.905720,1.000000,21.236842,58.000000,48.000000,4.000000,4.000000,0.000000,1.000000,7.000000,1.000000,1.000000,3.000000,141.097735,0.000000
max,200.000000,15778.000000,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000,1.000000,116.830643,88.000000,88.000000,16.000000,17.000000,1.000000,1.000000,13.000000,1.000000,1.000000,23.000000,170.457647,1.000000


In [11]:
## Vendo a média geral de gastos com saúde
df['health_expenditures'].mean()

17.03510284423828

In [12]:
## Vendo média por período
df.groupby('round')['health_expenditures'].mean()

round
0.0    17.210985
1.0    16.859217
Name: health_expenditures, dtype: float32

## Dividindo em Antes x Depois

Em um primeiro momento, só queremos domicílios que participaram do programa (enrolled == 1).

Problema: viés que decorrem de efeitos temporais estruturais ou de outras variáveis + viés de seleção (pessoas escolhem participar ou a seleção do organizador do programa pode não ser bem feita).

### Separação

In [13]:
## Dados de antes do programa
df_inscritos_antes = df.query('enrolled == 1 & round == 0')
df_inscritos_antes

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
0,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,0.0,15.185455,24.0,23.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
2,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,0.0,13.076257,30.0,26.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,0.0
5,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,0.0,15.286353,58.0,56.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,0.0
7,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,0.0,11.311761,35.0,24.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
8,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,0.0,11.223912,37.0,35.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11247,40.0,15773.0,1.0,1.0,1.0,1.0,1.0,28.979908,0.0,7.218568,57.0,51.0,3.0,0.0,0.0,1.0,9.0,1,1,1,114.763392,0.0
11249,40.0,15774.0,1.0,1.0,1.0,1.0,1.0,28.593508,0.0,7.241524,53.0,53.0,5.0,0.0,0.0,1.0,9.0,1,1,2,114.763392,0.0
11252,40.0,15775.0,1.0,1.0,1.0,1.0,1.0,37.171562,0.0,10.391406,29.0,27.0,1.0,6.0,0.0,1.0,6.0,1,1,0,114.763392,0.0
11253,40.0,15776.0,1.0,1.0,1.0,1.0,1.0,29.057186,0.0,8.679602,78.0,41.0,0.0,2.0,0.0,1.0,9.0,1,0,0,114.763392,0.0


In [14]:
## Dados de depois do programa
df_inscritos_depois = df.query('enrolled == 1 & round == 1')
df_inscritos_depois

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,25.0,24.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,31.0,27.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,59.0,57.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0
6,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,1.0,20.026909,36.0,25.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
9,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,1.0,16.664686,39.0,36.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11248,40.0,15773.0,1.0,1.0,1.0,1.0,1.0,28.979908,1.0,0.000000,58.0,52.0,3.0,0.0,0.0,1.0,9.0,1,1,1,114.763392,0.0
11250,40.0,15774.0,1.0,1.0,1.0,1.0,1.0,28.593508,1.0,7.193629,54.0,54.0,5.0,0.0,0.0,1.0,9.0,1,1,2,114.763392,0.0
11251,40.0,15775.0,1.0,1.0,1.0,1.0,1.0,37.171562,1.0,0.727366,30.0,28.0,1.0,6.0,0.0,1.0,6.0,1,1,0,114.763392,0.0
11254,40.0,15776.0,1.0,1.0,1.0,1.0,1.0,29.057186,1.0,0.000000,79.0,41.0,0.0,2.0,0.0,1.0,9.0,1,0,0,114.763392,0.0


In [15]:
## Dados de todos os inscritos no programa
df_inscritos = df.query('enrolled == 1')
df_inscritos

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
0,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,0.0,15.185455,24.0,23.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,25.0,24.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
2,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,0.0,13.076257,30.0,26.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,31.0,27.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,59.0,57.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11252,40.0,15775.0,1.0,1.0,1.0,1.0,1.0,37.171562,0.0,10.391406,29.0,27.0,1.0,6.0,0.0,1.0,6.0,1,1,0,114.763392,0.0
11253,40.0,15776.0,1.0,1.0,1.0,1.0,1.0,29.057186,0.0,8.679602,78.0,41.0,0.0,2.0,0.0,1.0,9.0,1,0,0,114.763392,0.0
11254,40.0,15776.0,1.0,1.0,1.0,1.0,1.0,29.057186,1.0,0.000000,79.0,41.0,0.0,2.0,0.0,1.0,9.0,1,0,0,114.763392,0.0
11255,40.0,15777.0,1.0,1.0,1.0,1.0,1.0,38.485317,1.0,0.000000,32.0,26.0,4.0,2.0,0.0,1.0,6.0,1,0,0,114.763392,0.0


### Comparando Médias de Gastos com Saúde

In [16]:
## Média total
df['health_expenditures'].mean()

17.03510284423828

In [17]:
## Média dos inscritos
df_inscritos['health_expenditures'].mean()

11.164377212524414

In [18]:
## Médias de antes
df_inscritos_antes['health_expenditures'].mean()

14.489691734313965

In [19]:
## Médias de depois
df_inscritos_depois['health_expenditures'].mean()

7.840174198150635

In [36]:
## Testando
# alternative: médias diferentes, 1ª maior ou 1ª menor?
stats.ttest_ind(df_inscritos_antes['health_expenditures'], df_inscritos_depois['health_expenditures'], nan_policy='omit', alternative='two-sided')

# P-valor MUITO baixo: rejeita-se a hipótese de que a média depois do programa é igual à de antes: média depois != média antes

Ttest_indResult(statistic=39.764560411867826, pvalue=9.90593763905028e-307)

In [37]:
## Testando
# alternative: médias diferentes, 1ª maior ou 1ª menor?
stats.ttest_ind(df_inscritos_antes['health_expenditures'], df_inscritos_depois['health_expenditures'], nan_policy='omit', alternative='less')

# P-valor MUITO alto: não podemos rejeitar a hipótese de que a média antes é menor que a média depois

Ttest_indResult(statistic=39.764560411867826, pvalue=1.0)

### Regressões

In [21]:
## Fazendo regressões simples
formula_simples = "health_expenditures ~ 1 + round"

modelo_simples_antesdepois = ols(formula_simples, df_inscritos).fit(cov_type="HC2", use_t=True)
print(modelo_simples_antesdepois.summary())

# Note: intercepto é a média de ANTES e round é a DIFERENÇA entre a média de depois e a de antes
# Assim, com o passar do tempo, os gastos (em dólares por pessoa por ano) cairam $ 6,65

                             OLS Regression Results                            
Dep. Variable:     health_expenditures   R-squared:                       0.211
Model:                             OLS   Adj. R-squared:                  0.210
Method:                  Least Squares   F-statistic:                     1582.
Date:                 Wed, 24 Nov 2021   Prob (F-statistic):          8.84e-307
Time:                         13:00:21   Log-Likelihood:                -19453.
No. Observations:                 5929   AIC:                         3.891e+04
Df Residuals:                     5927   BIC:                         3.892e+04
Df Model:                            1                                         
Covariance Type:                   HC2                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.4897      0.080    181.08

In [22]:
## Modelo mais completo
formula_completa = "health_expenditures ~ 1 + round + age_hh +  educ_hh + educ_sp + female_hh + indigenous + hhsize + dirtfloor + bathroom + land + hospital_distance"

modelo_completo_antesdepois = ols(formula_completa, df_inscritos).fit(cov_type="HC2", use_t=True)
print(modelo_completo_antesdepois.summary())

# Efeito de round é muito similar em ambas as regressões... por que?

                             OLS Regression Results                            
Dep. Variable:     health_expenditures   R-squared:                       0.483
Model:                             OLS   Adj. R-squared:                  0.482
Method:                  Least Squares   F-statistic:                     783.6
Date:                 Wed, 24 Nov 2021   Prob (F-statistic):               0.00
Time:                         13:00:21   Log-Likelihood:                -18201.
No. Observations:                 5929   AIC:                         3.643e+04
Df Residuals:                     5917   BIC:                         3.651e+04
Df Model:                           11                                         
Covariance Type:                   HC2                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            24.7391

## Comparando Inscritos e Não-Inscritos

### Separando

In [23]:
## Pegando apenas os dados no 2º período
df_depois = df.query('round == 1')
df_depois

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,25.0,24.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,31.0,27.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,59.0,57.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0
6,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,1.0,20.026909,36.0,25.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
9,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,1.0,16.664686,39.0,36.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19817,35.0,15731.0,0.0,0.0,0.0,0.0,0.0,74.980682,1.0,11.691319,47.0,39.0,5.0,3.0,0.0,1.0,7.0,0,0,2,162.748811,NaN
19819,35.0,15734.0,0.0,0.0,0.0,0.0,0.0,58.153015,1.0,13.734585,39.0,41.0,3.0,2.0,1.0,1.0,7.0,1,1,3,162.748811,NaN
19821,35.0,15738.0,0.0,0.0,0.0,0.0,0.0,59.737247,1.0,8.426427,53.0,41.0,0.0,2.0,0.0,1.0,7.0,0,1,2,162.748811,NaN
19824,40.0,15769.0,1.0,1.0,0.0,0.0,0.0,62.055641,1.0,8.248152,52.0,41.0,5.0,2.0,0.0,1.0,5.0,1,1,1,114.763392,NaN


In [24]:
## Vendo a proporção de inscritos ou não
df_depois['enrolled'].value_counts(normalize=True)

0.0    0.700928
1.0    0.299072
Name: enrolled, dtype: float64

### Médias

In [25]:
## Média de depois dos programas
df_depois["health_expenditures"].mean()

16.859230041503906

In [26]:
## Média dos inscritos
inscritos = df_depois.query("enrolled == 1")["health_expenditures"]
inscritos.mean()

7.840174198150635

In [27]:
## Média dos não-inscritos
naoinscritos = df_depois.query("enrolled == 0")["health_expenditures"]
naoinscritos.mean()

20.70746612548828

In [28]:
## Teste t
stats.ttest_ind(naoinscritos, inscritos, nan_policy='omit', alternative='greater')

# Rejeita-se H0 com força: a média dos inscritos é menor que a dos não inscritos
# (lembre-se: H0 é que a média dos não_inscritos é >= média dos inscritos)

Ttest_indResult(statistic=56.79261839104128, pvalue=0.0)

### Regressões

In [29]:
## Fazendo regressões simples
formula_simples = "health_expenditures ~ 1 + enrolled"

modelo_simples_inscritos_naoinscritos = ols(formula_simples, df_depois).fit(cov_type="HC2", use_t=True)
print(modelo_simples_inscritos_naoinscritos.summary())

# Note: intercepto é a média dos não-inscritos e enrolled é a diferença entre inscritos e não-inscritos
# Após o programa, domicílios inscritos gastam $12,86 a menor (por pessoa por ano)

                             OLS Regression Results                            
Dep. Variable:     health_expenditures   R-squared:                       0.246
Model:                             OLS   Adj. R-squared:                  0.245
Method:                  Least Squares   F-statistic:                     4188.
Date:                 Wed, 24 Nov 2021   Prob (F-statistic):               0.00
Time:                         13:00:22   Log-Likelihood:                -37215.
No. Observations:                 9914   AIC:                         7.443e+04
Df Residuals:                     9912   BIC:                         7.445e+04
Df Model:                            1                                         
Covariance Type:                   HC2                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     20.7075      0.134    154.44

In [30]:
## Modelo mais completo
formula_completa = "health_expenditures ~ 1 + enrolled + age_hh + age_sp +  educ_hh + educ_sp + female_hh + indigenous + hhsize + dirtfloor + bathroom + land + hospital_distance"

modelo_completo_inscritos_naoinscritos = ols(formula_completa, df_depois).fit(cov_type="HC2", use_t=True)
print(modelo_completo_inscritos_naoinscritos.summary())

# Coeficiente de enrolled já muda agora... qual a diferença pro outro caso?
# Programa provavelmente não é aleatório! (as características individuais parecem afetar o recebimento ou não do tratamento :/)

                             OLS Regression Results                            
Dep. Variable:     health_expenditures   R-squared:                       0.412
Model:                             OLS   Adj. R-squared:                  0.411
Method:                  Least Squares   F-statistic:                     496.8
Date:                 Wed, 24 Nov 2021   Prob (F-statistic):               0.00
Time:                         13:00:22   Log-Likelihood:                -35980.
No. Observations:                 9914   AIC:                         7.199e+04
Df Residuals:                     9901   BIC:                         7.208e+04
Df Model:                           12                                         
Covariance Type:                   HC2                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            29.2749

## Definindo Funções

In [31]:
## Função que faz os testes
def ols_diagnostics(formula, model, data, y_string):
    """
    Given the OLS model supplied, calculates statistics and draws graphs that check the Multiple Linear Regressions hypothesis.
    References:
        https://www.statsmodels.org/dev/examples/notebooks/generated/regression_diagnostics.html
        https://medium.com/@vince.shields913/regression-diagnostics-fa476b2f64db
    :param formula : patsy formula of the model;
    :param model : fitted model object;
    :param data : DataFrame containing the data; 
    :param y_string : string (name) of the dependent variable
    """

    ## Reset: specification of tue functional form of the model the model
    reset = linear_reset(model, power=3, use_f=True, cov_type='HC1')
    print(f"Linear Reset P-value: {reset.pvalue}")
    print("H0: model is well specificed and linear.")
    print("For more information, see the Residuals vs Fitted Values plot.\n")

    ### Condition number: multicollinearity
    print(f"Condition Number for Multicollinearity: {round(np.linalg.cond(model.model.exog), 2)}")
    print("The larger the number (> 10.000), the bigger the multicollinearity. For more information, see the 'VIF' plot.\n")

    ## Calculating Variance Influence Factors (VIF)
    # Matrices
    y, X = dmatrices(formula, data, return_type='dataframe')

    ## Calcuating VIFs and storing in a DataFrame
    dfVIF = pd.DataFrame()
    dfVIF["Variáveis"] = X.columns
    dfVIF["Fator_VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    ## Breusch-Pagan: 
    print(f"Breusch-Pagan P-value for heteroskedasticity: {round(sms.het_breuschpagan(model.resid, model.model.exog)[3], 4)}")
    print("H0: Variance is homoskedasticity.")
    print("For more information, see the 'Scale-Location' plot.\n")

    ## Durbin-Watson: correlation between the residuals
    print(f"Durbin-Watson statistic is: {np.around(durbin_watson(model.resid), 2)}")
    print("If the value is close to 0, there is positive serial correlation.")
    print("If the value is close to 4, there is negative serial correlation.")
    print("Rule of thumb: 1.5 < DW < 2.5 indicates no serial correlation.\n")
    
    ## Jarque-Bera: normality of the residuals (MLR 6, used for statistic inference)
    print(f"Jarque-Bera P-value: {np.around(sms.jarque_bera(model.resid)[1], 4)}")
    print("H0: Data has a normal distribution.")
    print("For more information, see the 'Normal Q-Q' plot.\n")

    ## Creating graphic object
    fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (12, 12))
    plt.style.use('seaborn-white')

    ### Plots
    ## Linearity: residuals x predicted values. The less inclined the lowess, the more linear the model.
    ax00 = sns.residplot(x=model.fittedvalues, y=y_string, data=data, lowess=True,
                         scatter_kws={'facecolors':'none', 'edgecolors':'black'},
                         line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8}, ax=ax[0, 0])
    
    # Titles
    ax00.set_title('Linearity: Residuals vs Fitted', fontsize=12)
    ax00.set_xlabel('Fitted Values', fontsize=10)
    ax00.set_ylabel('Residuals (horizontal lowess: linearity)', fontsize=10)

    ## Multicollinearity: VIF
    ax01 = dfVIF["Fator_VIF"].plot(kind='bar', stacked=False, ax=ax[0, 1])

    # X tick labels
    ax01.set_xticklabels(labels=dfVIF["Variáveis"], rotation=0, color='k')

    # Annotations
    for p in ax01.patches:                 
        ax01.annotate(round(p.get_height(), 2), (p.get_x()+p.get_width()/2., p.get_height()), 
                      ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    ## Titles
    ax01.set_title("Multicollinearity Test - VIF", color = 'k', fontsize=12)
    ax01.set_ylabel("Variance Influence Factor (> 5: multicollinearity)", color = 'k', fontsize=10)
    ax01.set_xlabel("Variable", color = 'k', fontsize=10)

    ## Heteroskedasticity: the more disperse and horizontal are the points, the more likely it is that homoskedasticity is present
    ax10 = sns.regplot(x=model.fittedvalues, y=np.sqrt(np.abs(model.get_influence().resid_studentized_internal)), 
                       scatter=True, ci=False,  lowess=True, line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
                       scatter_kws={'facecolors':'none', 'edgecolors':'black'}, ax=ax[1, 0])

    # Titles
    ax10.set_title('Heteroskedasticity: Scale-Location', fontsize=12)
    ax10.set_xlabel('Fitted Values', fontsize=10)
    ax10.set_ylabel('$\sqrt{|Standardized Residuals|}$ (disperse and horizontal: homoskedasticity)', fontsize=10)

    ## Normality of the residuals: Q-Q Plot
    probplot = sm.ProbPlot(model.get_influence().resid_studentized_internal, fit=True)
    ax11 = probplot.qqplot(line='45', marker='o', color='black', ax=ax[1, 1])

In [32]:
def cooks_distance(model):
    """
    Calculates and plots the Cooks Distance metric, which shows the influence of individual points in the regression results.
    If a point is above D = 0.5, then it affects the results and can be considered an outlier.
    References: https://medium.com/@vince.shields913/regression-diagnostics-fa476b2f64db
    :param model: fitted OLS model object.
    """

    ## Defining theme
    plt.style.use('seaborn-white')

    ## Creating functions that define D = 0.5 and D = 1.0
    def one_line(x):
        return np.sqrt((1 * len(model.params) * (1 - x)) / x)

    def point_five_line(x):
        return np.sqrt((0.5 * len(model.params) * (1 - x)) / x)
    
    def show_cooks_distance_lines(tx,inc,color,label):
        plt.plot(inc, tx(inc), label=label, color=color, ls='--')
    
    ## Plotting
    sns.regplot(x=model.get_influence().hat_matrix_diag, y=model.get_influence().resid_studentized_internal, 
                scatter=True, ci=False, lowess=True, line_kws={'color': 'blue', 'lw': 1, 'alpha': 0.8},
                scatter_kws={'facecolors':'none', 'edgecolors':'black'})
    
    show_cooks_distance_lines(one_line, np.linspace(.01,.14,100), 'red', 'Cooks Distance (D=1)' )

    show_cooks_distance_lines(point_five_line, np.linspace(.01,.14,100), 'black', 'Cooks Distance (D=0.5)')

    plt.title('Residuals vs Leverage', fontsize=12)
    plt.xlabel('Leverage', fontsize=10)
    plt.ylabel('Standardized Residuals', fontsize=10)
    plt.legend()